In [1]:
import torch
import torch.nn as nn
import numpy as np
import nibabel as nib

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

print("Path to dataset files:", path)

/Users/enesdemir/Desktop/BrainTumorModel/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/enesdemir/.cache/kagglehub/datasets/awsaf49/brats20-dataset-training-validation/versions/1


In [20]:
import os 
from torch.utils.data import Dataset, DataLoader
from torchio import Subject, ScalarImage, LabelMap

class BraTSDataset(Dataset):
    def __init__(self, folders, transform=None):
        self.transform = transform
        self.folders = folders
    
    def __len__(self):
        return len(self.folders)
    
    def __getitem__(self, idx):
        flair_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_flair.nii")
        t1_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t1.nii")
        t1ce_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t1ce.nii")
        t2_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t2.nii")
        seg_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_seg.nii")

        subject = Subject(
            flair=ScalarImage(flair_path),  
            t1ce=ScalarImage(t1ce_path),
            t2=ScalarImage(t2_path),
            seg=LabelMap(seg_path), 
            t1=ScalarImage(t1_path)
        )

        if self.transform:
            transformed_subject = self.transform(subject)

            input_tensor = torch.stack([
                transformed_subject.flair.data,
                transformed_subject.t1ce.data,
                transformed_subject.t2.data,
                transformed_subject.t1.data
            ], dim=0)  
            label = transformed_subject.seg.data  
        else:
            
            flair = torch.from_numpy(nib.load(flair_path).get_fdata()).float()
            t1ce = torch.from_numpy(nib.load(t1ce_path).get_fdata()).float()
            t2 = torch.from_numpy(nib.load(t2_path).get_fdata()).float()
            t1 = torch.from_numpy(nib.load(t1_path).get_fdata()).float()
            input_tensor = torch.stack([flair, t1ce,t2,t1], dim=0)
            label = torch.from_numpy(nib.load(seg_path).get_fdata()).float()

        return input_tensor, label

In [17]:
from sklearn.model_selection import train_test_split

path = '/Users/enesdemir/.cache/kagglehub/datasets/awsaf49/brats20-dataset-training-validation/versions/1'
real_path = os.path.join(path, 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData')

all_folders = [os.path.join(real_path,f) for f in os.listdir(real_path) if not f.endswith('.csv')]

train_folders, val_folders = train_test_split(all_folders, test_size=0.2, random_state=42)

Aşagıdaki data augmentiona araştırıcam

In [23]:
import torchio as tio

transform = tio.Compose([
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),  # Normalizasyon
    tio.RandomAffine(degrees=15, scales=(0.9, 1.1)),  # Augmentation
    tio.RandomNoise(std=0.1),  # Gürültü ekleme
])

In [ ]:
training_set = BraTSDataset(train_folders,transform=transform)
training_loader = DataLoader(training_set,
                             batch_size=4,
                             shuffle=True,
                             num_workers=0)

validation_set = BraTSDataset(val_folders,transform=transform)
validation_loader = DataLoader(validation_set,
                               batch_size=4,
                               shuffle=False,
                               num_workers=0)

for images, labels in training_loader:
    print(images.shape)
    print(labels.shape)
    break

for images, labels in validation_loader:
    print(images.shape)
    print(labels.shape)
    break


torch.Size([4, 4, 1, 240, 240, 155])
torch.Size([4, 1, 240, 240, 155])
torch.Size([4, 4, 1, 240, 240, 155])
torch.Size([4, 1, 240, 240, 155])
295
